In [1]:
import pandas as pd
from google.cloud import bigquery
import fasttext

In [2]:
country_id = 1

products_query = f"""
    SELECT * 
    FROM `peya-food-and-groceries.user_rodrigo_benitez.product_categories`
    WHERE country_id = {country_id}
    """

client = bigquery.Client()
products_ds = client.query(products_query).to_dataframe()

/home/rodrigobenitez/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/rodrigobenitez/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information 

In [3]:
# TO CSV

In [4]:
products_ds.sample(5)

,partner_id,partner_name,partner_description,business_type_id,business_type_name,country_id,legacyId_section,section_name,product_legacy_id,product_id,product_name,product_description,gtin,category_level_1_2_3
122223,291722,Mini Market Madina,None,2,Market,1,3484087,"Golosinas, alfajores y chocolates",60350772,60350772,Alfajor Oreo Trilogia 55 g,,00000077916495,Snacks | Confectionary | Alfajores
78447,389856,Supermercado Sam,None,2,Market,1,5711540,CUIDADO PERSONAL,100777159,100777159,Lotion Shea & Velver 400 Ml Love Beauty And Pl...,,00055086002363,Personal Care / Baby / Health | Personal Care ...
17569,286801,PedidosYa Market 11,None,2,Market,1,5461718,Aguas,59490573,59490573,Agua Nativa Sin Gas Sport 1 L,,07730130001938,Beverages | Water | Still
185305,368104,Kinko Treinta Y Tres,None,2,Market,1,4963461,Helados y Postres,90141175,90141175,Helado Conaprole Frutilla Durazno 1L,,07730105913006,Frozen | Ice Cream / Desserts | Ice Cream
179539,101888,NutriMarket - Av. Brasil,None,2,Market,1,792780,Almacén,6564098,6564098,Rollos De Cocina Sussex 2 Rollos,,07730219012541,Home / Pet | Cleaning / Laundry | Dishwashing


In [5]:
model_info = products_ds[['product_id', 'category_level_1_2_3', 'product_name']].fillna('')

In [6]:
model_info.sample(5)

,product_id,category_level_1_2_3,product_name
126628,72983612,BWS | Spirits | Whisky,Whisky Old Parr 12 Años 750ml
43874,49790306,Personal Care / Baby / Health | Personal Care ...,Antitranspirante Gillette Power Rush Clear Gel...
247997,105219912,Dairy / Chilled / Eggs | Dairy / Eggs | Cheese,Farming Queso Muzzarela Ligth Deslactosada Fet...
78085,59481828,BWS | Spirits | Ron,Ron Captain Morgan Original 750 mL
6558,66152053,Home / Pet | Cleaning / Laundry | Cleaning Pro...,Desinfectante Agua Jane Multisuperficies Gatil...


Proceso de limpieza de texto, aqui se modifican cosas como:
- se eliminan numeros
- se normaliza texto a lowercase
- se elimina simbología especial (Ñ, tildes, dieresis) con librería UNICODE
- eliminar UNIDADES de MEDIA ????? (tokenizar y ver palabras de 1,2 y 3 gramas)

In [7]:
import re
from unidecode import unidecode

def preprocess_text(text):
    word = re.sub('\n', ' ', text, re.UNICODE)
    word = unidecode(word)
    word = re.sub(r'[^\w\s]', '', word, re.UNICODE)
    word = re.sub(r"\b(\d+|[a-z])\b *", '', word, re.UNICODE)
    word = re.sub(r'[0-9]+', '', word)
#     word = re.sub(r'[0-9]+', '', word)
    word = word.lower()
    word = word.strip()
    return word

In [8]:
model_info['full_text'] = model_info['category_level_1_2_3'] + ' ' + model_info['product_name']
model_info['preprocessed_text'] = model_info['full_text'].apply(preprocess_text)

model_info.sample(5)

,product_id,category_level_1_2_3,product_name,full_text,preprocessed_text
157339,93056527,,Granola c/ miel 100Gr,Granola c/ miel 100Gr,granola miel gr
146571,92745999,Packaged Foods | Tea / Coffee | Tea,Te Verde Botica Del Señor 25 Saquitos,Packaged Foods | Tea / Coffee | Tea Te Verde B...,packaged foods tea coffee tea te verde boti...
243978,98333633,Dairy / Chilled / Eggs | Deli / Snacking | Meat,Centenario Bondiola Fetas 200Gr,Dairy / Chilled / Eggs | Deli / Snacking | Mea...,dairy chilled eggs deli snacking meat cen...
91174,98340285,Snacks | Confectionary | Occasional / Seasonal,Huevo Milka Mujer Maravilla 157G,Snacks | Confectionary | Occasional / Seasonal...,snacks confectionary occasional seasonal hu...
189274,56572331,Home / Pet | Disposables | Paper Products,Servilletas Papstar Roja,Home / Pet | Disposables | Paper Products Serv...,home pet disposables paper products servill...


In [10]:
model_info[['preprocessed_text']].to_csv('/home/rodrigobenitez/Documents/GitHub/Proyecto-Final-IMF/datasets/model_input.csv', index=False)

In [11]:
model = fasttext.train_unsupervised('/home/rodrigobenitez/Documents/GitHub/Proyecto-Final-IMF/datasets/model_input.csv')

Read 3M words
Number of words:  7542
Number of labels: 0
Progress: 100.0% words/sec/thread:  114897 lr:  0.000000 avg.loss:  0.834994 ETA:   0h 0m 0s
